<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/zaladowanie_repo_w_colabie/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 200 kB 49.0 MB/s 
     |████████████████████████████████| 199 kB 66.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=7a71f91695a6bfe2032bc01db950be80325e152d703039898c5711f28e930f7b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-17 12:44:38--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Załadowanie repozytorium

##Załadowanie repozytorium

In [4]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 294 (delta 81), reused 146 (delta 53), pack-reused 73
Receiving objects: 100% (294/294), 4.44 MiB | 20.75 MiB/s, done.
Resolving deltas: 100% (95/95), done.


##Testujemy dostępność do metod z katalogu tests

In [5]:
import sys

In [6]:
sys.path.append('/content/BigData2022-actors/tests')
import simple_test

In [7]:
simple_test.test_check_if_pytest_works()

#Pobranie danych

##Import danych

In [8]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-17 12:45:05--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.41, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236472510 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.52M  34.1MB/s    in 7.0s    

2022-11-17 12:45:13 (32.1 MB/s) - ‘name.basics.tsv.gz’ saved [236472510/236472510]

--2022-11-17 12:45:13--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.41, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287598573 (274M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.27M   127M

##Rozpakowanie danych

In [9]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [10]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [11]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00564...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [12]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [13]:
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [14]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (12081796, 5)
df_title_principals_selected dataframe size:  (20774832, 4)
df_title_basic_selected dataframe size:  (9376979, 5)
joined dataframe size:  (20771203, 8)
joined dataframe size:  (20771203, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [15]:
sample = data.rdd.takeSample(False, 5)

In [16]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,tconst,nconst,primaryName,birthYear,deathYear,knownForTitles,category,characters,titleType,originalTitle,isAdult,genres
0,tt1319362,nm1071709,Yôko Moriguchi,1966,\N,"tt1360082,tt0304874,tt8075192,tt0405821",actress,"[""Nami Shiraishi""]",tvEpisode,Ai no kon-yoku roten buro,0,"Comedy,Drama,Romance"
1,tt5028546,nm0600979,Brooks Moore,\N,\N,"tt5210554,tt0150338,tt3960750,tt1595607",actor,"[""Narrator""]",tvEpisode,"Time Delay Locks, Brownies, Pallet Dispensers,...",0,Documentary
2,tt9457678,nm0207344,Gelli de Belen,1973,\N,"tt0218821,tt7069178,tt0318673,tt0335036",actress,"[""Marcella Razon""]",tvEpisode,Episode #1.45,0,Drama
3,tt6727800,nm0342143,Antonio Grimau,1944,\N,"tt0349005,tt2121945,tt1375688,tt0451499",actor,"[""Marcelo Morales""]",tvEpisode,Episode #1.80,0,"Drama,Mystery,Romance"
4,tt14853146,nm3874608,Kushal Badrike,\N,\N,"tt20244998,tt5590026,tt6286358,tt7794358",actor,\N,tvEpisode,Episode #1.427,0,"Comedy,Talk-Show"
